In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [60]:
#Step 1: Load libraries for the U-net Model


import cv2
import datetime
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, TensorBoard
from tensorflow.keras import backend as keras
import tensorflow as tf

In [3]:
def conv_block(x, filter_size, size, stride, batch_norm=False, multiple=False):
    
    conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
    if multiple:
        conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
        conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
            
    if batch_norm is True:
        conv = layers.BatchNormalization()(conv)
    conv = layers.Activation("relu")(conv)
    
    return conv

In [24]:
def oleg_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    
    block_1 = conv_block(inputs1, 3, 64, 1, batch_norm=True)
    block_2 = conv_block(block_1, 3, 64, 1, batch_norm=True, multiple=True)
    block_3 = conv_block(block_2, 3, 128, 1, batch_norm=True)
    block_4 = conv_block(block_3, 3, 128, 1, batch_norm=True, multiple=True)
    block_5 = conv_block(block_4, 3, 256, 1, batch_norm=True)
    block_6 = conv_block(block_5, 3, 256, 1, batch_norm=True, multiple=True)
    block_7 = conv_block(block_6, 3, 512, 1, batch_norm=True)
    block_8 = conv_block(block_7, 3, 512, 1, batch_norm=True, multiple=True)
    
    average_pooling = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_8)
    flatten = tf.reshape(average_pooling,[-1])
    oleg_final = layers.Dense(1, activation = "linear")(average_pooling)
    
    model = models.Model(inputs1, oleg_final, name="Oleg_Model")
    return model

In [90]:
def lstm_block(x, size, multiple=False):
    
    conv = layers.LSTM(size, activation = 'relu', input_shape=(1500,2), kernel_initializer = 'he_normal')(x)
    if multiple:
        conv = layers.LSTM(size, activation = 'relu', input_shape=(1500,2), kernel_initializer = 'he_normal')(x)
        conv = layers.LSTM(size, activation = 'relu', input_shape=(1500,2), kernel_initializer = 'he_normal')(x)
    
    return conv

In [91]:
def lstm_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    block_1 = lstm_block(inputs1,  64)
    block_2 = lstm_block(block_1,  64, multiple=True)
    block_3 = lstm_block(block_2,  128)
    block_4 = lstm_block(block_3,  128, multiple=True)
    block_5 = lstm_block(block_4,  256)
    block_6 = lstm_block(block_5,  256, multiple=True)
    block_7 = lstm_block(block_6,  512)
    block_8 = lstm_block(block_7,  512, multiple=True)
    
    dense_1 = layers.Dense(1, activation = 'linear')(block_8)
    model = models.Model(inputs1, dense_1, name="Oleg_Model")
    return model
    

In [81]:
def multihead_oleg_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    
    block_1_a = conv_block(inputs1, 3, 64, 1, batch_norm=True)
    block_2_a = conv_block(block_1_a, 3, 64, 1, batch_norm=True, multiple=True)
    block_3_a = conv_block(block_2_a, 3, 128, 1, batch_norm=True)
    block_4_a = conv_block(block_3_a, 3, 128, 1, batch_norm=True, multiple=True)
    average_pooling_a = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_4_a)
    flatten_a = layers.Flatten()(average_pooling_a)
    
    block_1_b = conv_block(inputs1, 5, 64, 1, batch_norm=True)
    block_2_b = conv_block(block_1_b, 5, 64, 1, batch_norm=True, multiple=True)
    block_3_b = conv_block(block_2_b, 5, 128, 1, batch_norm=True)
    block_4_b = conv_block(block_3_b, 5, 128, 1, batch_norm=True, multiple=True)
    average_pooling_b = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_4_b)
    flatten_b = layers.Flatten()(average_pooling_b)
        
    block_1_c = conv_block(inputs1, 7, 64, 1, batch_norm=True)
    block_2_c = conv_block(block_1_c, 7, 64, 1, batch_norm=True, multiple=True)
    block_3_c = conv_block(block_2_c, 7, 128, 1, batch_norm=True)
    block_4_c = conv_block(block_3_c, 7, 128, 1, batch_norm=True, multiple=True)
    average_pooling_c = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_4_c)
    flatten_c = layers.Flatten()(average_pooling_c)
    
    flatten = tf.keras.layers.concatenate([flatten_a, flatten_b, flatten_c])
    oleg_final = layers.Dense(100, activation = "relu")(flatten)
    oleg_final = layers.Dense(1, activation = "relu")(oleg_final)
    
    model = models.Model(inputs1, oleg_final, name="Multihead_Model")
    return model

In [93]:
 model = oleg_model(input_shape = (2,1500))
# model = multihead_oleg_model(input_shape = (2,1500))
# model = lstm_model(input_shape = (2,1500))

In [83]:
model.summary()

Model: "Oleg_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 2, 1500)]         0         
                                                                 
 conv1d_200 (Conv1D)         (None, 2, 64)             288064    
                                                                 
 batch_normalization_100 (Ba  (None, 2, 64)            256       
 tchNormalization)                                               
                                                                 
 activation_100 (Activation)  (None, 2, 64)            0         
                                                                 
 conv1d_203 (Conv1D)         (None, 2, 64)             12352     
                                                                 
 batch_normalization_101 (Ba  (None, 2, 64)            256       
 tchNormalization)                                      

In [76]:
X_train = np.load('train_eptad_data.npy')
y_train = np.load('train_eptad_labels.npy')

In [80]:
np.quantile(X_train[:,1,:], [0,1])

array([-41.167292,  90.      ])

In [58]:
# Split Dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)


In [59]:
X_train.dtype, X_val.dtype, y_train.dtype, y_val.dtype

(dtype('float64'), dtype('float64'), dtype('int64'), dtype('int64'))

In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [41]:
?tf.keras.losses.binary_crossentropy

In [55]:
y_train[y_train == 0].shape[0] + y_train[y_train != 0].shape[0]

143988

In [78]:
y_train.shape, X_train.shape

((143988,), (143988, 2, 1500))

In [70]:
k = 3
dropout= 0
lr = 0.0003

In [84]:
# Checkpoints
model_name = 'OLEG_model_kernel_size=' + str(k) + '_dropout=' + str(dropout) + '_lr=' + str(lr)
model_checkpoint = ModelCheckpoint('D:/ETPAD.v2/models/' + model_name + '.hdf5', monitor='loss',verbose=1, save_best_only=True)

# Early Stop
early_stop = EarlyStopping(
    monitor='accuracy',
    min_delta=0.001,
    patience= 15,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

# Scheduler
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
scheduler_callback = LearningRateScheduler(scheduler)

# Tensorboard
log_dir = r"D:\ETPAD.v2\logs"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

In [85]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size = 32)